# Merge LoRA Adapter Weights and aply AWQ quantization

In [1]:
model_path = "/mnt/personal/mlynatom/thesis_models/it-Llama-3.1-8B-Instruct-full_mix_11_cs_en"

## 16 bit merge

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.


model_name = model_path.split("/")[-1]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"{model_path}/final",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.381 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
# Merge to 16bit - local
model.save_pretrained_merged(f"{model_path}/merge_16bit", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 626.84 out of 1007.71 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 84%|████████▍ | 27/32 [00:01<00:00, 31.89it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:04<00:00,  7.36it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [ ]:
# #merge # to 16bit - push to hub
# model.push_to_hub_merged(f"aic/model", tokenizer, save_method = "merged_16bit", token = "")

In [ ]:
# Merge to 4bit - local
# model.save_pretrained_merged(f"{model_path}/merge_4bit", tokenizer, save_method = "merged_4bit",)

In [ ]:
# merge to 4bit - push to hub
#model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

## AWQ

In [ ]:
#%pip install autoawq

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_id = f"{model_path}/merge_16bit"
quant_path = f"{model_path}/awq"
quant_config = { "zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM" }

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

# Save quantized model
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

print(f'Model is quantized and saved at "{quant_path}"')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
AWQ: 100%|██████████| 32/32 [09:26<00:00, 17.70s/it]


Model is quantized and saved at "/mnt/personal/mlynatom/thesis_models/it-Llama-3.1-8B-Instruct-full_mix_11_cs_en/awq"
